In [1]:
using PlotlyJS
using Images
# using Revise
using DDA

In [2]:
using StaticArrays
using GeometryBasics

In [4]:
# target: Au disk
# grid spacing: d = 0.2 nm
# d = 10 nm
# # of dipoles along the diameter: 50
# gap between two disks: d = (1) 0.2 nm, (2) 0.4 nm, (5) 1 nm,  (100) 20 nm

# # of dipoles: ??
# >> peak resonance at ??? n

# steps:
# 1. create the coordinates of the dipoles,
# 2. assign the polarizability αj to each dipole,
# 3. calculated the incident field E_inc, at each dipole,
# 4. assemble the interaction matrix A and
# 5. solve for P in the system of linear equations

In [5]:
# Future:
# - calculate the E field on an extended grid (which contains empty space)
#   - extending the ogidinal grid where we already have the polarisation values on the dipoles
#   - redo FFT convolutioon to calcluster the field in an efficient way

In [6]:

# 1. Define a grid
# disk_r = 5. # nm
disk_d = 10. # nm
disk_h = 1. # nm
disk_gap = 1. # nm
# gap: d = (1) 0.2 nm, (2) 0.4 nm, (5) 1 nm,  (100) 20 nm

spacing = .2

Nx = round(Int, (2 * disk_d + disk_gap) / spacing)
Ny = round(Int, disk_d / spacing)
Nz = round(Int, disk_h / spacing) 

offset = [spacing / 2, spacing / 2, spacing / 2]
origin = [-(2 * disk_d + disk_gap) / 2, -disk_d / 2 , -disk_h / 2] .+ offset

grid = CartesianGrid(origin, [spacing, spacing, spacing], (Nx, Ny, Nz))
grid


105×50×5 CartesianGrid{Float64,3}
  minimum: [-10.4, -4.9, -0.4]
  maximum: [10.4, 4.9, 0.4]
  spacing: [0.2, 0.2, 0.2]

In [ ]:
# 2. Define the target(s)
origin = [(disk_d + disk_gap) / 2, 0, 0] 

d1 = DDA.Disk(-origin, disk_d / 2, disk_h)
occ1 = DDA.discretize(grid, d1);


In [ ]:
layout = Layout(yaxis=attr(scaleanchor="x", scaleratio=1))
plot(heatmap(z=occ1[:,:,1]' .* 1, colorscale = "Viridis"), layout)

In [ ]:
d2 = DDA.Disk(origin, disk_d / 2, disk_h)
occ2 = DDA.discretize(grid, d2);

occ = occ1 .|| occ2

layout = Layout(yaxis=attr(scaleanchor="x", scaleratio=1))
plot(heatmap(z=occ[:,:,1]' .* 1, colorscale = "Viridis"), layout)

In [ ]:
d1 = DDA.Disk(-origin, disk_d / 2, disk_h)
d2 = DDA.Disk(origin, disk_d / 2, disk_h)
s = DDA.Composite([d1, d2])

occ = DDA.discretize(grid, s) 

layout = Layout(yaxis=attr(scaleanchor="x", scaleratio=1))
plot(heatmap(z=occ[:,:,1]' .* 1, colorscale = "Viridis"), layout)

In [ ]:

# 1. Define a grid
# disk_r = 5. # nm
disk_d = 10. # nm
disk_h = 1. # nm
disk_gap = 4/3 # nm

spacing = 1/3

Nx = round(Int, (2 * disk_d + disk_gap) / spacing)
Ny = round(Int, disk_d / spacing)
Nz = round(Int, disk_h / spacing) 

offset = [spacing / 2, spacing / 2, spacing / 2]
origin = [-(2 * disk_d + disk_gap) / 2, -disk_d / 2 , -disk_h / 2] .+ offset

grid = CartesianGrid(origin, [spacing, spacing, spacing], (Nx, Ny, Nz))
grid


In [ ]:
# 2. Define the target(s)
origin = [(disk_d + disk_gap) / 2, 0, 0] 
radius = disk_d / 2

d1 = DDA.Disk(-origin, disk_d / 2, disk_h)
occ1 = DDA.discretize(grid, d1);

d2 = DDA.Disk(origin, disk_d / 2, disk_h)
occ2 = DDA.discretize(grid, d2);

occ = occ1 .|| occ2

layout = Layout(yaxis=attr(scaleanchor="x", scaleratio=1))
plot(heatmap(z=occ[:,:,1]' .* 1, colorscale = "Viridis", xgap=3, ygap=3), layout)

In [ ]:
coords = grid[occ]
layout = Layout(scene=attr(aspectmode=:data, camera_projection_type=:orthographic))


trace = scatter3d(x=[p[1] for p in coords], y=[p[2] for p in coords], z=[p[3] for p in coords], marker=attr(color=1:length(coords)), mode="markers")
plot(trace, layout)

In [ ]:
ε = 1.33 + 0.1im
model = DDA.LDRModel(ε)
scatterer = DDA.Scatterer(s, model)

# s1 = DDA.Scatterer(d1, model1)
# s2 = DDA.Scatterer(d2, model2)
# s = DDA.Composite([s1, s2])

In [ ]:
using StaticArrays

In [ ]:
SVector{3, Float64} <: StaticVector{3, Float64}

In [ ]:
StaticVector{3, Float64}

In [ ]:
Point3{Float64} <: SVector{3, Float64}

In [ ]:
typeof(grid[1])

In [ ]:
supertype(supertype(Point3{Float64} ))

In [ ]:
field(f::PlaneWave, r) = f.E₀ * exp(im * dot(f.kvec, r))


In [ ]:
# 4. Define incindent field
k = 2π      # wavenumber
e = [1, 0]  # Jones polarisation vector
θ, ϕ = 0., 0. # rotation angles [rad]

E_inc = DDA.PlaneWave(k, e, θ, ϕ)
E = DDA.field(E_inc, grid[occ])

In [ ]:
# incident field
E0 = 1
k = [0, 0, 1]
e = [1, 0, 0]
E_inc = E_inc()


# grid and target
r = 22
d = 2*r/30

s = Sphere(r, [0,0,0])
g = CubicGrid(-r:d:r,-r:d:r,-r:d:r)

# get_dipoles
# dipoles = get_dipoles(g, s)
dipoles = positions(g, s)

# material model

# solve
lambda = 550.
# lambda = 400:10:600
solve(dipoles, E_inc)

??material